In [ ]:
# 裁判文书工具地址：!git clone https://github.com/IanHongruZhang/Openlaw_tools

In [ ]:
### 引入所有包,如果缺少某个包，包的名字附于之后
import numpy as np
# numpy

import pandas as pd
# pandas

import requests
# requests

from bs4 import BeautifulSoup
# BeautifulSoup

import re
# re

from fake_useragent import UserAgent
# fake-useragent

import json
# json

import time
# time

import random
# random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# selenium

from PIL import Image,ImageEnhance
# PIL

import hashlib
# hashlib

from collections import Counter
# collections

from gensim import corpora, models, similarities
# gensim 

import codecs, sys
# codecs

import os
# os

import time
#time

import shutil
# shutil

import jieba
# jieba

from pyhanlp import *
# pyhanlp，注意hanlp需要java的工具

import jpype
# jpype

import tensorflow as tf
# tensorflow

import matplotlib.pyplot as plt
# matplotlib

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
# sklearn

from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFParser, PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# pdfminer3k

In [ ]:
def import_table():
    """
    return table"选中的表格
    """
    try:
        root = Tk()
        root.filename =  filedialog.askopenfilename(filetypes=(("xlsx", "*.xlsx"),("xlsx", "*.xlsx")))        
        if ".xlsx" in root.filename:
            ### 该目录下有该文件
            table = pd.read_excel(root.filename)
            root.withdraw()
            return table
    except Exception as e:
        root.withdraw()
        print("导入错误")

def save_model(table):
    print("-------------------------------------------")
    print("以下为表格的预览:")
    print(table.head())
    flag = input("是否需要保存该词频表？(输入Y/N):")
    try:
        if flag == "Y":
            save_file_name = input("请输入该词频表的名称(不用加.xlsx):")
            save_file_name_xlsx = save_file_name + ".xlsx"
            table.to_excel(save_file_name_xlsx)
        else:
            pass
    except Exception as e:
        pass


### 第一步：API导入文书
-----------------------------------------------------------------------
* 此步借助openlaw API的平台来得到文书：http://develop.openlaw.cn/index.jsp


* 个人账号文书的每日上限是1000份文书，因此请控制下列代码运行时的条件。

In [ ]:
### API查询模块，最终得到的是一个含有案件号的excel文件
def extract(html_list):
    list_cases = list(map(lambda x:x.text,html_list))
    list_cases_2 = list(map(lambda x:eval(x)["judgements"],list_cases))
    cases_3 = list(np.array(list_cases_2).flatten())
    list_case = []
    for index in range(0,10):
        for item in cases_3[index]:
            elem = item["judgement"]
            list_case.append(elem)
    return list_case

class openlaw():
    def __init__(self):
        self.AppKey = "xxx"
        self.AppSecret = "xxx"
        self.Nonce = str("xxx")
        self.Curtime = str(int(time.time() * 1000))
    
    def openlaw_api_headers(self):
        self.Combined = self.AppSecret + self.Nonce + self.Curtime
        sha1 = hashlib.sha1(self.Combined.encode("utf-8"))
        self.CheckSum = sha1.hexdigest()
        self.headers = {"AppKey":self.AppKey,
                   "Nonce":self.Nonce,
                   "CurTime":self.Curtime,
                   "CheckSum":self.CheckSum}
        return self.headers

    def openlaw_api_search(self):
        target = "http://develop.openlaw.cn/judgement/search"
        keyword = input("请输入需要查询的案例关键词：")
        url = target + "?keyword=" + keyword
        ### 以下为选填参数
        """
        请求参数说明
        参数	参数说明
        keyword	搜索关键字
        court	法院名称
        judgeDateYear	判决年份
        """
        switch = True
        while switch == True:
            court,year = None,None
            paramater_none = input("是否需要其他设定参数（输入Y/N）")
            if paramater_none == "Y":
                pass
            else:
                break
            paramater = input("请输入还需要限定的参数")
            if paramater == "法院名称":
                court = input("哪个法院？")
                url = url + "&court=" + str(court)
            if paramater == "判决年份":
                year = input("哪一年?")
                url = url + "&judgeDateYear=" + str(year)
            paramater_none = input("还需要设定参数吗？（输入Y/N）")
            if paramater_none == "N":
                switch = False
            else:
                switch = True
        headers = self.openlaw_api_headers()
        for index in range(0,11):
            url = url + "&judgeDateBegin=2011-03-01" # 需要修改的地方
            url = url + "&judgeDateEnd=2019-03-01" # 需要修改的地方
            url = url + "&page=10"
        html = requests.get(url,headers = headers)
        return html
    
    def openlaw_api_search_v2(self,keyword):
        #### 设定法院进行爬取
        target = "http://develop.openlaw.cn/judgement/search"
        list_urls = []
        url = target + "?keyword=" + keyword
        with open("courts.txt","r",encoding = "utf-8") as f:
            court_list = eval(f.read())
        #### 选择你需要的法院
        for court in court_list[0:10]:
            url_single = url + "&court=" + str(court)
            list_urls.append(url_single)
        headers = self.openlaw_api_headers()
        html_list = list(map(lambda x:requests.get(x,headers = headers),list_urls))
        list_case = extract(html_list)
        return list_case
        
    def openlaw_api_court(self):
        target = "http://develop.openlaw.cn/judgement/court"
        dicts = {}
        keycourt = input("请输入需要查询的法院：")
        headers = self.openlaw_api_headers()
        dicts["court"] = keycourt
        html = requests.get(target,headers = headers,data = data)
        return html
        
    def openlaw_api_analysis(self):
        target = "http://develop.openlaw.cn/analytic/judgement"
        dicts = {}
        keycourt = input("请输入需要查询的关键字：")
        dicts["keyword"] = keyword
        html = requests.get(target,headers = headers,data = data)
        return html
    
    def openlaw_api_case_detail(self):
        target = "http://develop.openlaw.cn/judgement/detail"
        dicts = {}
        keycourt = input("")
    
o1 = openlaw()
html = o1.openlaw_api_search()